In [17]:
from pymongo import MongoClient
from datetime import datetime

def insert_data():
    client = MongoClient('mongodb://localhost:27017/')
    db = client.local  # 'local' 데이터베이스 사용

    # 책 데이터 삽입
    books = [
        {"title": "Kafka on the Shore", "author": "Haruki Murakami", "year": 2002},
        {"title": "Norwegian Wood", "author": "Haruki Murakami", "year": 1987},
        {"title": "1Q84", "author": "Haruki Murakami", "year": 2009}
    ]
    db.books.insert_many(books)

    # 영화 데이터 삽입
    movies = [
        {"title": "Inception", "director": "Christopher Nolan", "year": 2010, "rating": 8.8},
        {"title": "Interstellar", "director": "Christopher Nolan", "year": 2014, "rating": 8.6},
        {"title": "The Dark Knight", "director": "Christopher Nolan", "year": 2008, "rating": 9.0}
    ]
    db.movies.insert_many(movies)

    # 사용자 행동 데이터 삽입
    user_actions = [
        {"user_id": 1, "action": "click", "timestamp": datetime(2023, 4, 12, 8, 0)},
        {"user_id": 1, "action": "view", "timestamp": datetime(2023, 4, 12, 9, 0)},
        {"user_id": 2, "action": "purchase", "timestamp": datetime(2023, 4, 12, 10, 0)},
    ]
    db.user_actions.insert_many(user_actions)

    print("Data inserted successfully")
    client.close()

if __name__ == "__main__":
    insert_data()


 

Data inserted successfully


In [ ]:
from pymongo import MongoClient

client = MongoClient('mongodb://localhost:27017/')
db = client.local  

# 장르 추가
db.books.update_many(
    {},  
    {"$set": {"genre": "fantasy"}}  
)

print("Genre field added to all books")
client.close()


Genre field added to all books


**[문제 1: 특정 장르의 책 찾기]**

- **문제 설명**:
데이터베이스에 새로운 필드로 **`genre`**를 책 데이터에 추가하였습니다. 사용자는 "fantasy" 장르의 모든 책을 찾고 싶어합니다.
- **쿼리 작성 목표**:
"fantasy" 장르에 해당하는 모든 책의 제목과 저자를 찾는 MongoDB 쿼리를 함수로 만들어 문제를 해결해 봅니다.

In [27]:
from pymongo import MongoClient

client = MongoClient('mongodb://localhost:27017/')
db = client.local 

def find_books_by_genre(db, genre):
    query = {"genre": genre}
    projection = {"_id": 0, "title": 1, "author": 1}
    books = db.books.find(query, projection)
    for book in books:
        print(book)

find_books_by_genre(db, 'fantasy')

{'title': 'Kafka on the Shore', 'author': 'Haruki Murakami'}
{'title': 'Norwegian Wood', 'author': 'Haruki Murakami'}
{'title': '1Q84', 'author': 'Haruki Murakami'}
{'title': 'Kafka on the Shore', 'author': 'Haruki Murakami'}
{'title': 'Norwegian Wood', 'author': 'Haruki Murakami'}
{'title': '1Q84', 'author': 'Haruki Murakami'}
{'title': 'Kafka on the Shore', 'author': 'Haruki Murakami'}
{'title': 'Norwegian Wood', 'author': 'Haruki Murakami'}
{'title': '1Q84', 'author': 'Haruki Murakami'}
{'title': 'Kafka on the Shore', 'author': 'Haruki Murakami'}
{'title': 'Norwegian Wood', 'author': 'Haruki Murakami'}
{'title': '1Q84', 'author': 'Haruki Murakami'}
{'title': 'Kafka on the Shore', 'author': 'Haruki Murakami'}
{'title': 'Norwegian Wood', 'author': 'Haruki Murakami'}
{'title': '1Q84', 'author': 'Haruki Murakami'}
{'title': 'Kafka on the Shore', 'author': 'Haruki Murakami'}
{'title': 'Norwegian Wood', 'author': 'Haruki Murakami'}
{'title': '1Q84', 'author': 'Haruki Murakami'}
{'title': 

**[문제 2: 감독별 평균 영화 평점 계산]**

- **문제 설명**:
각 영화 감독별로 그들의 영화 평점의 평균을 계산하고 싶습니다. 이를 통해 어떤 감독이 가장 높은 평균 평점을 가지고 있는지 알아볼 수 있습니다.
- **쿼리 작성 목표**: 
모든 영화 감독의 영화 평점 평균을 계산하고, 평균 평점이 높은 순으로 정렬하는 MongoDB 쿼리를 함수로 만들어 문제를 해결해 봅니다.

In [28]:
def calculate_average_ratings(db):
    movies_collection = db.movies
    pipeline = [
        {"$group": {"_id": "$director", "average_rating": {"$avg": "$rating"}}},
              
        {"$sort": {"average_rating": -1}},
        
        # 결과 포맷 조정 (director, average_rating만 출력)
        {"$project": {"_id": 0, "director": "$_id", "average_rating": 1}}
    ]

    # Aggregation 실행
    results = movies_collection.aggregate(pipeline)
    
    # 결과 출력
    for result in results:
        print(f"Director: {result['director']}, Average Rating: {result['average_rating']}")

# 함수 실행
calculate_average_ratings(db)


Director: Christopher Nolan, Average Rating: 8.8


**[문제 3: 특정 사용자의 최근 행동 조회]**

- **문제 설명**:
특정 사용자의 최근 행동 로그를 조회하고자 합니다. 이 때, 최신 순으로 정렬하여 최근 5개의 행동만을 보고 싶습니다.
- **쿼리 작성 목표**:
사용자 ID가 1인 사용자의 최근 행동 5개를 시간 순으로 정렬하여 조회하는 MongoDB 쿼리를 함수로 만들어 문제를 해결해 봅니다.

In [ ]:
def find_recent_actions_by_user(db, user_id, limit=5):
    user_actions_collection = db.user_actions
    query = {"user_id": user_id}
    sort_criteria = {"timestamp": -1}  # 최신 순으로 정렬

    actions = user_actions_collection.find(query).sort(sort_criteria).limit(limit)
    for action in actions:
        print(action)

# 함수 실행
find_recent_actions_by_user(db, 1)

{'_id': ObjectId('67e4fd0dd7bb0a97b2d1c64c'), 'user_id': 1, 'action': 'view', 'timestamp': datetime.datetime(2023, 4, 12, 9, 0)}
{'_id': ObjectId('67e4fcc4d7bb0a97b2d1c641'), 'user_id': 1, 'action': 'view', 'timestamp': datetime.datetime(2023, 4, 12, 9, 0)}
{'_id': ObjectId('67e4fd1dd7bb0a97b2d1c660'), 'user_id': 1, 'action': 'view', 'timestamp': datetime.datetime(2023, 4, 12, 9, 0)}
{'_id': ObjectId('67e4fd14d7bb0a97b2d1c656'), 'user_id': 1, 'action': 'view', 'timestamp': datetime.datetime(2023, 4, 12, 9, 0)}
{'_id': ObjectId('67e4f89c7b82684317fc0bf2'), 'user_id': 1, 'action': 'view', 'timestamp': datetime.datetime(2023, 4, 12, 9, 0)}


**[문제 4: 출판 연도별 책의 수 계산]**

- **문제 설명** :
데이터베이스에 저장된 책 데이터를 이용하여 각 출판 연도별로 출판된 책의 수를 계산하고자 합니다. 이 데이터는 시간에 따른 출판 트렌드를 분석하는 데 사용될 수 있습니다.
- **쿼리 작성 목표** :
각 출판 연도별로 출판된 책의 수를 계산하고, 출판된 책의 수가 많은 순서대로 정렬하는 MongoDB 쿼리를 함수로 만들어 문제를 해결해 봅니다.

In [30]:
def count_books_by_year(db):
    books_collection = db.books
    pipeline = [
        {"$group": {"_id": "$year", "count": {"$sum": 1}}},  
        {"$sort": {"count": -1}},  
        {"$project": {"_id": 0, "year": "$_id", "count": 1}}  # 결과 포맷 조정
    ]

    results = books_collection.aggregate(pipeline)
    for result in results:
        print(f"Year: {result['year']}, Book Count: {result['count']}")

# 함수 실행 
count_books_by_year(db)


Year: 2002, Book Count: 7
Year: 1987, Book Count: 7
Year: 2009, Book Count: 7


**[문제 5: 특정 사용자의 행동 유형 업데이트]**

- **문제 설명**:
특정 사용자의 행동 로그 중, 특정 날짜 이전의 "view" 행동을 "seen"으로 변경하고 싶습니다. 예를 들어, 사용자 ID가 1인 사용자의 2023년 4월 13일 이전의 모든 "view" 행동을 "seen"으로 변경하는 작업입니다.
- **쿼리 작성 목표**:
사용자 ID가 1인 사용자의 2023년 4월 13일 이전의 "view" 행동을 "seen"으로 변경하는 MongoDB 업데이트 쿼리를 함수로 만들어 문제를 해결해 봅니다.

In [31]:
from datetime import datetime

def update_user_actions_before_date(db, user_id, date, old_action, new_action):
    user_actions_collection = db.user_actions
    query = {"user_id": user_id, "action": old_action, "timestamp": {"$lt": date}}
    update = {"$set": {"action": new_action}}

    result = user_actions_collection.update_many(query, update)
    print(f"Updated {result.modified_count} documents.")

# 함수 실행
update_user_actions_before_date(db, 1, datetime(2023, 4, 13), "view", "seen")

Updated 7 documents.
